In [2]:
import gym
import torch
import random
import math
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time

# if gpu is to be used
use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
Tensor = torch.Tensor
LongTensor = torch.LongTensor

env = gym.make('CartPole-v1')



seed_value = 23
env.seed(seed_value)
torch.manual_seed(seed_value)
random.seed(seed_value)

###### PARAMS ######
learning_rate = 0.001
num_episodes = 500
gamma = 0.9999

hidden_layer = 64

replay_memory_size = 50000
batch_size = 32

double_dqn = True

update_target_frequency = 500

egreedy = 0.9 # pourcentage exploration/exploitation
egreedy_final = 0.01 # finir l'algo avec ce pourcentage exploration/exploitation
egreedy_decay = 500 #modification du eggreey par episode

report_interval = 10
score_to_solve = 195

clip_error = False
####################

number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n


def calculate_epsilon(steps_done):
    epsilon = egreedy_final + (egreedy - egreedy_final) * \
              math.exp(-1. * steps_done / egreedy_decay)
    return epsilon

class ExperienceReplay(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, state, action, new_state, reward, done):
        transition = (state, action, new_state, reward, done)

        if self.position >= len(self.memory):
            self.memory.append(transition)
        else:
            self.memory[self.position] = transition

        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
       return zip(*random.sample(self.memory, batch_size))

    def __len__(self):
        return len(self.memory)



class NeuralNetwork(nn.Module):
    def __init__(self):
        # super(NeuralNetwork, self).__init__()
        super().__init__()
        self.linear1 = nn.Linear(number_of_inputs, hidden_layer)
        self.linear2 = nn.Linear(hidden_layer, number_of_outputs)

        self.activation = nn.Tanh()
        #self.activation = nn.ReLU()


    def forward(self, x):
        output1 = self.linear1(x)
        output1 = self.activation(output1)
        output2 = self.linear2(output1)

        return output2


class QNet_Agent(object):
    def __init__(self):
        self.nn = NeuralNetwork().to(device)
        self.target_nn = NeuralNetwork().to(device)

        self.loss_func = nn.MSELoss()
        # self.loss_func = nn.SmoothL1Loss()

        self.optimizer = optim.Adam(params=self.nn.parameters(), lr=learning_rate)
        # self.optimizer = optim.RMSprop(params=mynn.parameters(), lr=learning_rate)

        self.update_target_counter = 0

    def select_action(self, state, epsilon):

        random_for_egreedy = torch.rand(1)[0]

        if random_for_egreedy > epsilon:

            with torch.no_grad():

                state = Tensor(state).to(device)
                action_from_nn = self.nn(state)
                action = torch.max(action_from_nn, 0)[1]
                action = action.item()
        else:
            action = env.action_space.sample()

        return action

    def optimize(self):

        if (len(memory) < batch_size):
            return

        state, action, new_state, reward, done = memory.sample(batch_size)

        state = Tensor(state).to(device)
        new_state = Tensor(new_state).to(device)

        reward = Tensor(reward).to(device)
        action = LongTensor(action).to(device)
        done = Tensor(done).to(device)


        if double_dqn:
            new_state_indexes = self.nn(new_state).detach()
            max_new_state_indexes = torch.max(new_state_indexes, 1)[1]

            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = new_state_values.gather(1,max_new_state_indexes.unsqueeze(1)).squeeze(1)

        else:
            new_state_values = self.target_nn(new_state).detach()
            max_new_state_values = torch.max(new_state_values, 1)[0]

        target_value = reward + (1-done) * gamma * max_new_state_values


        predicted_value = self.nn(state).gather(1, action.unsqueeze(1)).squeeze(1)

        loss = self.loss_func(predicted_value, target_value)

        self.optimizer.zero_grad()
        loss.backward()

        if clip_error:
            for param in self.nn.parameters():
                param.grad.data.clamp_(-1,1)

        self.optimizer.step()

        if self.update_target_counter % update_target_frequency == 0:
            self.target_nn.load_state_dict(self.nn.state_dict())

        self.update_target_counter


        # Q[state, action] = reward + gamma * torch.max(Q[new_state])

memory = ExperienceReplay(replay_memory_size)
qnet_agent = QNet_Agent()

steps_total = []

frames_total = 0
solved_after = 0
solved = False

start_time = time.time()

for i_episode in range(num_episodes):

    state = env.reset()

    step = 0
    # for step in range(100):
    while True:

        step += 1
        frames_total += 1

        epsilon = calculate_epsilon(frames_total)

        # action = env.action_space.sample()
        action = qnet_agent.select_action(state, epsilon)

        new_state, reward, done, info = env.step(action)

        memory.push(state, action, new_state, reward, done)
        qnet_agent.optimize()

        state = new_state
        if done:
            steps_total.append(step)
            mean_reward_100 = sum(steps_total[-100:])/100

            if mean_reward_100 > score_to_solve and solved == False:
                print("Solved after : "+str(i_episode))
                solved_after = i_episode
                solved = True

            if i_episode % report_interval == 0:
                print("*** Episode : " + str(i_episode)+" ***")
                print("Average reward : " + str())
                print("Sum total last 10 steps : " +str(sum(steps_total[-report_interval:])))
                print("Average of last 100 steps : " + str(mean_reward_100))
                print("Average of all steps : " + str(sum(steps_total)/len(steps_total)))
                print("Epsilon : " +str(epsilon))
                print("Frames total : " + str(frames_total))
                elapsed_time = time.time() - start_time
                print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
                print()

            break

print("Average reward: %.2f" % (sum(steps_total) / num_episodes))
print("Average reward (last 100 episodes): %.2f" % (sum(steps_total[-100:]) / 100))

if solved :
    print("Solved after : " + str(solved_after))
plt.figure(figsize=(12, 5))
plt.title("Rewards")
plt.bar(torch.arange(len(steps_total)), steps_total, alpha=0.6, color='green', width=5)
plt.show()

env.close()
env.env.close()



C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\gym\utils\passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\gym\core.py:201: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  "Function `env.seed(seed)` is marked as deprecated and will be removed in the future. "
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\ipykernel_launcher.py:139: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorc

*** Episode : 0 ***
Average reward : 
Sum total last 10 steps : 32
Average of last 100 steps : 0.32
Average of all steps : 32.0
Epsilon : 0.8448244495823493
Frames total : 32
00:00:00

*** Episode : 10 ***
Average reward : 
Sum total last 10 steps : 230
Average of last 100 steps : 2.62
Average of all steps : 23.818181818181817
Epsilon : 0.5370110218906584
Frames total : 262
00:00:00

*** Episode : 20 ***
Average reward : 
Sum total last 10 steps : 304
Average of last 100 steps : 5.66
Average of all steps : 26.952380952380953
Epsilon : 0.29692517363542875
Frames total : 566
00:00:00

*** Episode : 30 ***
Average reward : 
Sum total last 10 steps : 454
Average of last 100 steps : 10.2
Average of all steps : 32.903225806451616
Epsilon : 0.12572555268179905
Frames total : 1020
00:00:01

*** Episode : 40 ***
Average reward : 
Sum total last 10 steps : 305
Average of last 100 steps : 13.25
Average of all steps : 32.31707317073171
Epsilon : 0.07287957962378233
Frames total : 1325
00:00:02

**

KeyboardInterrupt: 